## Finding hospitals with expensive lab tests

Let's clone the database and export the data (this might take a sec ;-))

In [ ]:
!sudo bash -c 'curl -L https://github.com/dolthub/dolt/releases/latest/download/install.sh | sudo bash'
!dolt clone dolthub/quest-v3
!dolt sql -q "select billing_code_type, billing_code, billing_code_modifier, reporting_entity_name, negotiated_rate, npi from rate join npi_rate on npi_rate.rate_id = rate.id join code on code.id = rate.code_id join price_metadata on price_metadata.id = rate.price_metadata_id join insurer on insurer.id = rate.insurer_id" -r csv >> outputwnpi.csv

Our analysis is done in polars, which (imo) has a cleaner API than pandas.

In [44]:
import polars as pl
from polars import col

In [93]:
save_dir = './quest-v3-redux' # change this to '.' if file saved in this dir

In [46]:
df = pl.read_csv(f'{save_dir}/outputwnpi.csv', infer_schema_length = 10_000)

In [47]:
df = df.unique()

In [69]:
placeholder_prices = [ # "suspected..."
    999999.99,         # Sierra Health...
    699999.99,         # Blue Cross
    99999.99,          # UMR
    88888.88,          # United, Medica, Oxford
    49999.5,
    39999.6,           # Rocky Mountain Health placeholder value
    8720.0,            # Aetna
    811.0,             # Anthem (?)
    458.0,             # Anthem (?)
    140.0,             # ?
    .01,               # Aetna (?) (internal?)
    .02,               # (?)
    0]

df = df.filter(~col('negotiated_rate').is_in(placeholder_prices))

There's one billing code that consistently comes up as confusingly expensive in this analysis and I'm not sure why. It's a simple blood draw coded CPT 36416 or 36415, and it's usually bundled with other codes, and not billed separately. I'm going to filter it out for the time being.

In [70]:
df = df.filter(~col('billing_code').is_in(['36416', '36415']))

In [71]:
def compute_means_and_ratios(df) -> pl.DataFrame():
    """Compute the mean of each negotiated rate to get a kind of reference value.
    The 'multiplier' is the negotiated_rate/mean."""
    return (df
      .with_column(
          pl.mean('negotiated_rate').over(['billing_code_type', 'billing_code', 'billing_code_modifier']).alias('rate_mean')
      ).with_column(
          (col('negotiated_rate')/col('rate_mean')).alias('multiplier')
      ))

In [72]:
df = compute_means_and_ratios(df)

Let's get rid of any prices that are too low which might be skewing our mean downwards. This makes our analysis more robust -- by making the average price as high as reasonably-is-possible, we can say more confidently that prices that are way higher than this are truly outliers.

In [73]:
df = df.filter(col('multiplier') > .01)

We'll need to compute the means and ratios again.

In [74]:
df = compute_means_and_ratios(df)

Now let's look at hospitals which appear often in this dataset. We'll filter down to rates which are over 20x the average.

In [75]:
(df
 .filter(col('multiplier') > 20) # filter down to the highest negotiated rates
 .select(['npi', 'billing_code_type', 'billing_code', 'billing_code_modifier',])
 .unique()
 ['npi']                         # get just the NPI numbers
 .value_counts()
 .sort('counts')                 # sort by the NPIs that appear most frequently in this set
 [-10:]                          # take just the last 10
)

npi,counts
i64,u32
1831140979,9
1649273434,9
1285798918,9
1699770149,9
1326079534,10
1154618742,10
1881697878,13
1902897820,15
1194706655,47


Let's make this easier to understand by joining this with NPPES, the database of NPIs with provider information.

In [ ]:
!wget https://download.cms.gov/nppes/NPPES_Data_Dissemination_January_2023.zip
!unzip NPPES_Data_Dissemination_January_2023.zip

In [56]:
npi = pl.scan_csv(f'{save_dir}/npidata_pfile_20050523-20230108.csv', infer_schema_length = 10_000)

In [57]:
npi = npi.select(['NPI', 
            'Entity Type Code',
            'Provider Organization Name (Legal Business Name)', 
            'Provider First Line Business Practice Location Address', 
            'Provider Business Practice Location Address City Name', 
            'Provider Business Practice Location Address State Name',
            'Healthcare Provider Taxonomy Code_1',])

In [58]:
npi = npi.collect()

In [88]:
exp_hosps = (df
 .filter(col('multiplier') > 20) # filter down to the highest negotiated rates
 .select(['npi', 'billing_code_type', 'billing_code', 'billing_code_modifier',])
 .unique()
 ['npi']                         # get just the NPI numbers
 .value_counts()
 .sort('counts')                 # sort by the NPIs that appear most frequently in this set
 [-20:]                          # take just the last 10
).join(npi, left_on = 'npi', right_on = 'NPI').sort('counts').rename({'counts': 'number_disinct_codes_gt_20_times_mean_rate'})

In [92]:
exp_hosps

npi,number_disinct_codes_gt_20_times_mean_rate,Entity Type Code,Provider Organization Name (Legal Business Name),Provider First Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Healthcare Provider Taxonomy Code_1
i64,u32,str,str,str,str,str,str
1780620526,5,"""2""","""QUEST DIAGNOST...","""33608 ORTEGA H...","""SAN JUAN CAPIS...","""CA""","""291U00000X"""
1447296272,5,"""2""","""QUEST DIAGNOST...","""14225 NEWBROOK...","""CHANTILLY""","""VA""","""291U00000X"""
1992285282,7,"""2""","""NHCI OF HILLSB...","""101 CIRCLE DR""","""HILLSBORO""","""TX""","""282NC0060X"""
1235685892,7,"""2""","""FAIRFIELD HOSP...","""125 NEWMAN ST""","""FAIRFIELD""","""TX""","""282N00000X"""
1871911016,7,"""2""","""NORTH TEXAS - ...","""3101 N TARRANT...","""FORT WORTH""","""TX""","""282N00000X"""
1316197767,8,"""2""","""CRANE COUNTY H...","""1310 S ALFORD ...","""CRANE""","""TX""","""282NC0060X"""
1386950350,8,"""2""","""CRANE COUNTY H...","""1310 S ALFORD ...","""CRANE""","""TX""","""282NC0060X"""
1326564071,8,"""2""","""MULTICARE HEAL...","""17700 SE 272ND...","""COVINGTON""","""WA""","""282N00000X"""
1740273994,8,"""2""","""ECTOR COUNTY H...","""500 W 4TH ST""","""ODESSA""","""TX""","""282N00000X"""


The last hospital, Havasu Regional, has the highest number of lab tests with a cost ratio of over 20x the mean price. We can look more closely at those rates by filtering down the first dataframe.

In [91]:
exp_npi = exp_hosps[-1]['npi'][0]
df.filter(col('npi') == exp_npi).sort('multiplier')

billing_code_type,billing_code,billing_code_modifier,reporting_entity_name,negotiated_rate,npi,rate_mean,multiplier
str,str,str,str,f64,i64,f64,f64
"""MS-DRG""","""793""",null,"""Blue Cross Blu...",6683.0,1780761866,50902.318082,0.131291
"""MS-DRG""","""794""",null,"""Blue Cross Blu...",2571.13,1780761866,17579.860579,0.146254
"""MS-DRG""","""807""",null,"""Blue Cross Blu...",2604.26,1780761866,9128.340628,0.285294
"""MS-DRG""","""807""",null,"""Blue Cross Blu...",2655.33,1780761866,9128.340628,0.290889
"""MS-DRG""","""795""",null,"""Blue Cross Blu...",831.0,1780761866,2663.575074,0.311987
"""MS-DRG""","""795""",null,"""Blue Cross Blu...",831.99,1780761866,2663.575074,0.312358
"""MS-DRG""","""806""",null,"""Blue Cross Blu...",3191.0,1780761866,9989.427309,0.319438
"""MS-DRG""","""769""",null,"""Blue Cross Blu...",8741.6,1780761866,22655.591391,0.385847
"""MS-DRG""","""557""",null,"""Blue Cross Blu...",7621.67,1780761866,19738.672194,0.386129
